In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Projects/2. Generative deep finance/Market_Scenario_Generator_Hackathon_starting_kit

/content/drive/MyDrive/Projects/2. Generative deep finance/Market_Scenario_Generator_Hackathon_starting_kit


In [3]:
import warnings
warnings.filterwarnings('ignore')

# Take stock price

In [4]:
import yfinance as yahooFinance

In [5]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 300)

import pickle
from datetime import datetime

import matplotlib.pyplot as plt

plt.style.use('classic')


# US stocks

In [6]:
ticker = [
    'AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD',
    'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK',
    'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'VZ', 'WBA', 'WMT', 'DIS'
]

In [16]:

%%time
stock = dict()

for t in ticker:
    info = yahooFinance.Ticker(t)
    stock_price = info.history(start = '2000-01-01', end = '2023-12-31')

    stock_price = stock_price[['Open', 'High', 'Low', 'Close']]
    stock_price.index = stock_price.index.tz_localize(None)

    stock_price['Daily_ret(t)'] = np.log(stock_price['Close']/stock_price['Close'].shift(1))
    stock_price['Volatility(t)'] = stock_price['Daily_ret(t)'].rolling(window = 7).std()


    stock_price = stock_price.dropna()
    stock_price.columns = [col + '_' + t for col in stock_price.columns]
    stock_price.columns = [col.upper() for col in stock_price.columns]

    #stock_price = stock_price[stock_price.index >= datetime.strptime('2000-01-01', '%Y-%m-%d')]
    stock[t] = stock_price


CPU times: user 3.8 s, sys: 57.3 ms, total: 3.85 s
Wall time: 9.01 s


In [18]:

len_df = pd.DataFrame()
len_df.index = ticker
for i in ticker:
    len_df.loc[i, 'Num'] = len(stock[i])
    len_df.loc[i, 'First date'] = stock[i].index[0]

print(len(len_df))
len_df.sort_values(by = 'Num')


27


,Num,First date
AXP,6030.0,2000-01-12
WBA,6030.0,2000-01-12
VZ,6030.0,2000-01-12
UNH,6030.0,2000-01-12
TRV,6030.0,2000-01-12
PG,6030.0,2000-01-12
NKE,6030.0,2000-01-12
MSFT,6030.0,2000-01-12
MRK,6030.0,2000-01-12
MMM,6030.0,2000-01-12


In [19]:

df = pd.DataFrame()

for i in ticker:
    if i == ticker[0]:
        df = stock[i]
    else:
        df = df.join(stock[i])


In [20]:
df.head()

,OPEN_AXP,HIGH_AXP,LOW_AXP,CLOSE_AXP,DAILY_RET(T)_AXP,VOLATILITY(T)_AXP,OPEN_AMGN,HIGH_AMGN,LOW_AMGN,CLOSE_AMGN,DAILY_RET(T)_AMGN,VOLATILITY(T)_AMGN,OPEN_AAPL,HIGH_AAPL,LOW_AAPL,CLOSE_AAPL,DAILY_RET(T)_AAPL,VOLATILITY(T)_AAPL,OPEN_BA,HIGH_BA,LOW_BA,CLOSE_BA,DAILY_RET(T)_BA,VOLATILITY(T)_BA,OPEN_CAT,HIGH_CAT,LOW_CAT,CLOSE_CAT,DAILY_RET(T)_CAT,VOLATILITY(T)_CAT,OPEN_CSCO,HIGH_CSCO,LOW_CSCO,CLOSE_CSCO,DAILY_RET(T)_CSCO,VOLATILITY(T)_CSCO,OPEN_CVX,HIGH_CVX,LOW_CVX,CLOSE_CVX,DAILY_RET(T)_CVX,VOLATILITY(T)_CVX,OPEN_GS,HIGH_GS,LOW_GS,CLOSE_GS,DAILY_RET(T)_GS,VOLATILITY(T)_GS,OPEN_HD,HIGH_HD,LOW_HD,CLOSE_HD,DAILY_RET(T)_HD,VOLATILITY(T)_HD,OPEN_HON,HIGH_HON,LOW_HON,CLOSE_HON,DAILY_RET(T)_HON,VOLATILITY(T)_HON,OPEN_IBM,HIGH_IBM,LOW_IBM,CLOSE_IBM,DAILY_RET(T)_IBM,VOLATILITY(T)_IBM,OPEN_INTC,HIGH_INTC,LOW_INTC,CLOSE_INTC,DAILY_RET(T)_INTC,VOLATILITY(T)_INTC,OPEN_JNJ,HIGH_JNJ,LOW_JNJ,CLOSE_JNJ,DAILY_RET(T)_JNJ,VOLATILITY(T)_JNJ,OPEN_KO,HIGH_KO,LOW_KO,CLOSE_KO,DAILY_RET(T)_KO,VOLATILITY(T)_KO,OPEN_JPM,HIGH_JPM,LOW_JPM,CLOSE_JPM,DAILY_RET(T)_JPM,VOLATILITY(T)_JPM,OPEN_MCD,HIGH_MCD,LOW_MCD,CLOSE_MCD,DAILY_RET(T)_MCD,VOLATILITY(T)_MCD,OPEN_MMM,HIGH_MMM,LOW_MMM,CLOSE_MMM,DAILY_RET(T)_MMM,VOLATILITY(T)_MMM,OPEN_MRK,HIGH_MRK,LOW_MRK,CLOSE_MRK,DAILY_RET(T)_MRK,VOLATILITY(T)_MRK,OPEN_MSFT,HIGH_MSFT,LOW_MSFT,CLOSE_MSFT,DAILY_RET(T)_MSFT,VOLATILITY(T)_MSFT,OPEN_NKE,HIGH_NKE,LOW_NKE,CLOSE_NKE,DAILY_RET(T)_NKE,VOLATILITY(T)_NKE,OPEN_PG,HIGH_PG,LOW_PG,CLOSE_PG,DAILY_RET(T)_PG,VOLATILITY(T)_PG,OPEN_TRV,HIGH_TRV,LOW_TRV,CLOSE_TRV,DAILY_RET(T)_TRV,VOLATILITY(T)_TRV,OPEN_UNH,HIGH_UNH,LOW_UNH,CLOSE_UNH,DAILY_RET(T)_UNH,VOLATILITY(T)_UNH,OPEN_VZ,HIGH_VZ,LOW_VZ,CLOSE_VZ,DAILY_RET(T)_VZ,VOLATILITY(T)_VZ,OPEN_WBA,HIGH_WBA,LOW_WBA,CLOSE_WBA,DAILY_RET(T)_WBA,VOLATILITY(T)_WBA,OPEN_WMT,HIGH_WMT,LOW_WMT,CLOSE_WMT,DAILY_RET(T)_WMT,VOLATILITY(T)_WMT,OPEN_DIS,HIGH_DIS,LOW_DIS,CLOSE_DIS,DAILY_RET(T)_DIS,VOLATILITY(T)_DIS
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2000-01-12,32.629042,33.243697,32.105932,32.472111,-0.005222,0.022524,48.271015,48.360241,44.657381,45.371185,-0.044238,0.072184,0.717125,0.720900,0.652961,0.658151,-0.061847,0.052090,27.675018,28.522212,27.432962,27.796045,0.004364,0.027294,13.523234,13.837344,13.523234,13.655491,0.017094,0.029858,36.238616,36.450537,35.094238,35.200199,-0.025558,0.039952,17.665981,17.780119,17.425024,17.602571,-0.003596,0.022418,60.657832,62.009789,60.117049,60.838093,-0.005171,0.037706,37.378245,37.681824,36.846980,37.074665,-0.008155,0.038924,32.606120,32.812053,32.434509,32.640442,0.009508,0.023957,61.751658,62.977657,61.041870,61.687134,0.004193,0.026399,25.975001,27.082600,25.510523,26.082188,0.017271,0.043944,24.949065,25.264666,24.766349,24.849401,-0.009314,0.028614,15.570304,15.826342,15.474290,15.746330,0.011242,0.029871,22.864750,23.254374,22.803231,23.049309,0.006247,0.017478,22.655551,23.831564,22.482608,23.658621,0.041797,0.022423,18.833414,19.145187,18.614017,18.614017,0.002485,0.037744,29.073842,29.690860,28.999802,29.394693,0.019500,0.043172,33.510519,33.626339,32.255805,32.680477,-0.033114,0.022750,4.903394,4.915441,4.734727,4.758822,-0.034829,0.041045,30.540475,30.998746,30.409540,30.638676,-0.006390,0.036285,17.910811,18.250031,17.775124,18.080421,0.001878,0.020765,6.016010,6.129045,5.978331,6.009730,-0.009360,0.045705,16.268612,16.629335,16.160395,16.160395,-0.034283,0.022330,17.935627,18.159822,17.748798,18.085091,0.020878,0.029935,14.284570,14.405969,14.041773,14.041773,-0.018086,0.036728,27.715676,27.715676,25.691227,25.980434,-0.073313,0.071176
2000-01-13,32.746744,33.112920,32.275944,32.642120,0.005222,0.015616,46.218830,48.360243,45.549639,47.691051,0.049867,0.064115,0.713233,0.745432,0.698254,0.730335,0.104069,0.068065,27.796042,27.997755,27.110218,27.352274,-0.016094,0.028819,13.771219,14.118393,13.771219,13.804283,0.010837,0.028319,35.941905,36.111443,35.178987,36.005482,0.0

In [21]:
# https://core.ac.uk/download/pdf/52391988.pdf

# df.to_pickle('./data/df.pkl')


In [58]:
data = df.filter(like = '(T)')

In [59]:
label = pd.DataFrame(index = data.index)
label['LABEL'] = 0

label.loc[(label.index >= datetime.strptime('2008-10-01', '%Y-%m-%d')) \
            & (label.index <= datetime.strptime('2009-03-01', '%Y-%m-%d')) , 'LABEL'] = 1

label.loc[(label.index >= datetime.strptime('2020-03-01', '%Y-%m-%d')) \
            & (label.index <= datetime.strptime('2020-09-01', '%Y-%m-%d')) , 'LABEL'] = 1


In [60]:
fig, ax = plt.subplots(nrows = len(ticker), ncols = 2, figsize = (30, 100))

for i in range(len(ticker)):
    ax[i, 0].plot(data.iloc[:, 2*i], label = f'PRICE', color = 'blue', linewidth = 1)
    ax2 = ax[i, 0].twinx()
    ax2.plot(label['LABEL'], color = 'red', alpha = 0.5, linewidth = 2)
    ax2.set_ylim([-0.1, 1.1])
    # ax[i, 0].legend(loc = 'upper right')

    ax[i, 1].plot(data.iloc[:, 2*i - 1], label = f'VOLATILITY', color = 'blue', linewidth = 1)
    ax3 = ax[i, 1].twinx()
    ax3.plot(label['LABEL'], color = 'red', alpha = 0.5, linewidth = 2)
    ax3.set_ylim([-0.1, 1.1])
    # ax[i, 1].legend(loc = 'upper right')

plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [62]:
def split_sequences(sequences, n_step_ppaths):
    X = list()
    for i in range(len(sequences)):
    # find the end of this pattern
        end_ix = i + n_step_ppaths
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x = sequences[i:end_ix, :]
        X.append(seq_x)
    return np.array(X)

In [70]:
training_data = data[data.index < datetime.strptime('2012-01-01', '%Y-%m-%d')]
training_label = label[label.index < datetime.strptime('2012-01-01', '%Y-%m-%d')]

valid_data = data[data.index >= datetime.strptime('2012-01-01', '%Y-%m-%d')]
valid_label = label[label.index >= datetime.strptime('2012-01-01', '%Y-%m-%d')]

In [72]:
training_data_split = split_sequences(training_data.values, n_step_ppaths = 5)
training_label = training_label.iloc[4:, ].values

valid_data_split = split_sequences(valid_data.values, n_step_ppaths = 5)
valid_label = valid_label.iloc[4:, ].values

In [73]:
training_data_split.shape, training_label.shape, valid_data_split.shape, valid_label.shape,

((3008, 5, 54), (3008, 1), (3014, 5, 54), (3014, 1))

In [74]:
with open('./data/training_data.pkl', 'wb') as file:
    pickle.dump(training_data_split, file)

with open('./data/training_label.pkl', 'wb') as file:
    pickle.dump(training_label, file)

with open('./data/valid_data.pkl', 'wb') as file:
    pickle.dump(valid_data_split, file)

with open('./data/valid_label.pkl', 'wb') as file:
    pickle.dump(valid_label, file)


In [76]:
training_label.shape, sum(training_label), valid_label.shape, sum(valid_label)

((3008, 1), array([103]), (3014, 1), array([129]))